## Job Seeker Assistant Chatbot


*This project implements an AI-powered conversational assistant designed to help tech industry job seekers. Using the DialoGPT model, the chatbot provides personalized guidance for resume building, cover letter writing, and interview preparation. The implementation focuses on creating a user-friendly interface that maintains conversation context while delivering industry-specific advice and templates.*

### Environment Setup

In [1]:
# Importing the required libraries
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
import random
import re
import ipywidgets as widgets
from IPython.display import display, clear_output
import time
from collections import deque
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Checking CUDA availability for GPU acceleration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"This device will use {device}.")

This device will use cpu.


In [3]:
# Setting random seed for reproducibility
random.seed(52)
torch.manual_seed(52)

In [4]:
# Configuring the basic parameters for the Chatbot
MAX_LENGTH = 100  
TEMPERATURE = 0.8  
TOP_P = 0.9

### Model Initialization

In [5]:
MODEL_NAME = "microsoft/DialoGPT-medium"

In [6]:
# Initializing the model and tokenizer
def initialize_model():
    print(f"Loading {MODEL_NAME}....")

    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
        model = model.to(device)
        print("Model loaded successfully!")
        return model, tokenizer

    except Exception as e:
        print(f"Error loading model: {e}")
        return None, None
    

In [7]:
# Calling the function to initialize the model and tokenizer
model, tokenizer = initialize_model()

Loading microsoft/DialoGPT-medium....
Model loaded successfully!


In [8]:
# Setting the pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [9]:
# Configuring generation parameters
generation_config = {
    "max_length": MAX_LENGTH,
    "temperature": TEMPERATURE,
    "top_p": TOP_P,
    "top_k": 50,
    "repetition_penalty": 1.2,
    "no_repeat_ngram_size": 2,
    "pad_token_id": tokenizer.pad_token_id,
    "eos_token_id": tokenizer.eos_token_id
}

print("Model initialization complete with the following generation parameters:")
for param, value in generation_config.items():
    print(f"- {param}: {value}")

Model initialization complete with the following generation parameters:
- max_length: 100
- temperature: 0.8
- top_p: 0.9
- top_k: 50
- repetition_penalty: 1.2
- no_repeat_ngram_size: 2
- pad_token_id: 50256
- eos_token_id: 50256


### Conversation Engine

In [10]:
# Function to process user input and generate a response
def get_response(user_input, chat_history_ids = None):
    # Encoding the user input to get input IDs
    user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors = "pt")
    user_input_ids = user_input_ids.to(device)

    # If chat history exists, combining it with new user input
    if chat_history_ids is not None:
        bot_input_ids = torch.cat([chat_history_ids, user_uinput_ids], dim = 1)
    else:
        bot_input_ids = user_input_ids

    attention_mask = torch.ones(bot_input_ids.shape, dtype=torch.long, device=device)
    
    # Generating a response
    chat_history_ids = model.generate(
        bot_input_ids,
        attention_mask=attention_mask,
        max_length=1000,  # Maximum length of the entire conversation
        pad_token_id=tokenizer.pad_token_id,
        no_repeat_ngram_size=generation_config["no_repeat_ngram_size"],
        do_sample=True,
        top_p=generation_config["top_p"],
        top_k=generation_config["top_k"],
        temperature=generation_config["temperature"],
        repetition_penalty=generation_config["repetition_penalty"]
    )

    response = chat_history_ids[0, bot_input_ids.shape[-1]:]

    # Decoding the response
    response_text = tokenizer.decode(response, skip_special_tokens = True)

    return response_text, chat_history_ids

In [11]:
# Function to handle the conversation flow
def chat():
    print("Job Seeker Assistant Chatbot initialized.")
    print("Hi there!\nHow can I help you with your job search today? (Resume building, Cover Letter, or Interview Preparation)")
    
    chat_history_ids = None
    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "end", "bye", "quit"]:
            print("Bot: Thank you for using the Job Seeker Assistant. Good luck with your job search!")
            break 
        try:
            response, chat_history_ids = get_response(user_input, chat_history_ids)
            pritn(f"Bot: {response}")

        except Exception as e:
            print(f"An error occured: {e}")
            chat_history_ids = None

In [12]:
# # Testing the conversation engine
# def test_response(user_input):
#     print(f"You: {user_input}")
#     response, _ = get_response(user_input)
#     print(f"Bot: {response}")
#     #return response

In [13]:
# # Test with a sample input
# sample_input = "Hi! I want to build a resume."
# test_response(sample_input)

### Context Management System

In [14]:
class ConversationManager:
    def __init__(self, tokenizer, max_history_tokens = 800):
        """
        Initializes the Conversation Manager

        Args:
        tokenizer: The tokenizer used by the model
        max_history_tokens: Maximum number of tokens to keep in history with some buffer space
        """
        self.tokenizer = tokenizer
        self.max_history_tokens = max_history_tokens
        self.conversation_history = deque(maxlen=20)
        self.token_history = []
        self.topic = None

    def add_exchange(self, user_input, bot_response):
        """
        Adds a conversation exchange to history
        """
        exchange = {"user": user_input, "bot": bot_response}
        self.conversation_history.append(exchange)

        # Updating token history with both user input and bot response
        user_tokens = self.tokenizer.encode(user_input + self.tokenizer.eos_token)
        bot_tokens = self.tokenizer.encode(bot_response + self.tokenizer.eos_token)

        # Adding new tokens to history
        self.token_history.extend(user_tokens)
        self.token_history.extend(bot_tokens)

        # Trimming token history if it exceed the maximum length
        if len(self.token_history) > self.max_history_tokens:
            self.token_history = self.token_history[-self.max_history_tokens:]

    def get_token_history(self):
        """
        Getting the token history for model input
        """
        if not self.token_history:
            return None

        return torch.tensor([self.token_history], dtype = torch.long).to(device)

    def detect_topic(self, user_input):
        """Detect the topic of conversation based on user input"""
        user_input_lower = user_input.lower()

        # Check for resume-related keywords
        if any(word in user_input_lower for word in ["resume", "cv", "experience", "skill", "qualification"]):
            self.topic = "Resume"

        # Check for cover letter-related keywords
        elif any(word in user_input_lower for word in ["cover letter", "application letter", "introduction letter"]):
            self.topic = "Cover Letter"

        # Check for interview-related keywords
        elif any(word in user_input_lower for word in ["interview", "question", "answer", "behavioral", "technical"]):
            self.topic = 'Interview'

    def get_conversation_summary(self):
        """
        Getting a summary of the conversation history
        """
        if not self.conversation_history:
            return "No conversation history yet."

        summary = f"Current topic - {self.topic if self.topic else "General"}\n"
        summary += f"Conversation length - {len(self.conversation_history)} exchanges\n"

        # Adding the last exchange
        if self.conversation_history:
            last_exchange = self.conversation_history[-1]
            summary += f"\nLast User Query: {last_exchange["user"]}\n"
            summary += f"Last response: {last_exchange['bot']}"

        return summary

    def clear_history(self):
        """
        Clearing the conversation history
        """
        self.conversation_history.clear()
        self.token_history = []
        self.topic = None

# Function to process user input and generate a response using the conversation manager
def get_contextual_response(conversation_manager, user_input, model):
    """
    Generating a response based on user input and conversation history
        
    Args:
        conversation_manager: The conversation manager containing history
        user_input: The current user input
        model: The language model
        
    Returns:
        bot_response: The generated response
    """ 
    conversation_manager.detect_topic(user_input)
    new_user_input_ids = tokenizer.encode(user_input + tokenizer.eos_token, return_tensors = "pt").to(device)
    chat_history_ids = conversation_manager.get_token_history()

    # If chat history exists combininh it with new user input
    if chat_history_ids is not None:
        if chat_history_ids.dim() == 1:
            chat_history_ids = chat_history_ids.unsqueeze(0)

        bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim = -1)
    else:
        bot_input_ids = new_user_input_ids

    # Creating attention mask
    attention_mask = torch.ones(bot_input_ids.shape, dtype = torch.long, device = device)

    # Generating a response
    output_ids = model.generate(
        bot_input_ids,
        attention_mask = attention_mask,
        max_length = bot_input_ids.shape[-1] + 100,
        pad_token_id = tokenizer.pad_token_id,
        no_repeat_ngram_size = 2,
        do_sample = True,
        top_p = TOP_P,
        top_k = 40,
        temperature = TEMPERATURE,
        repetition_penalty = 1.2 
    )

    response_ids = output_ids[0, bot_input_ids.shape[-1]:]
    response = tokenizer.decode(response_ids, skip_special_tokens = True)

    conversation_manager.add_exchange(user_input, response)
    
    return response

In [15]:
# # Testing the conversation manager
# conversation_manager = ConversationManager(tokenizer)

# # Example conversation
# responses = []

# # First conversation
# user_input1 = "I need help with my resume for a software engineering job."
# print(f"User: {user_input1}")
# response1 = get_contextual_response(conversation_manager, user_input1, model)
# print(f"Bot: {response1}")
# responses.append(response1)

# # Follow-up questions
# user_input2 = "What should I include in my skills section?"
# print(f"User: {user_input2}")
# response2 = get_contextual_response(conversation_manager, user_input2, model)
# print(f"Bot: {response2}")
# responses.append(response2)

# # Printing the conversation summary
# print("\nConversation Summary:")
# print(conversation_manager.get_conversation_summary())

### Resume Module Implementation

In [16]:
class ResumeAssistant:
    def __init__(self):
        # Initializing resume templates
        self.resume_templates = {
            "chronological": {
                "description": "Traditional format focusing on work history in reverse chronological order.",
                "sections": ["Contact Information", "Professional Summary", "Work Experience", "Skills", "Education"]
            },
            "functional": {
                "description": "Skills-based format that emphasizes capabilities rather than work history.",
                "sections": ["Contact Information", "Professional Summary", "Skills & Expertise", "Accomplishments", "Work Experience", "Education"]
            },
            "combination": {
                "description": "Hybrid format that highlights both skills and work experience.",
                "sections": ["Contact Information", "Professional Summary", "Core Competencies", "Professional Experience", "Education"]
            },
            "technical": {
                "description": "Specialized format for tech roles, emphasizing technical skills and projects.",
                "sections": ["Contact Information", "Professional Summary", "Technical Skills", "Projects", "Work Experience", "Education", "Certifications"]
            }
        }
        
        # ATS keywords for different tech roles
        self.ats_keywords = {
            "software_engineer": [
                "Python", "Java", "JavaScript", "C++", "C#", "SQL", "Git", "CI/CD", "Docker", "Kubernetes",
                "REST API", "Cloud Services", "AWS", "Azure", "Google Cloud", "Agile", "Scrum", "TDD",
                "Data Structures", "Algorithms", "OOP", "Full-stack", "Backend", "Frontend", "Microservices"
            ],
            "data_scientist": [
                "Python", "R", "SQL", "Machine Learning", "Deep Learning", "TensorFlow", "PyTorch", "Keras",
                "Data Analysis", "Data Visualization", "Statistics", "A/B Testing", "Big Data", "Hadoop",
                "Spark", "NLP", "Computer Vision", "Predictive Modeling", "Feature Engineering", "Clustering"
            ],
            "ux_designer": [
                "User Research", "Wireframing", "Prototyping", "User Testing", "Figma", "Sketch", "Adobe XD",
                "Information Architecture", "User Flows", "Usability", "Accessibility", "Design Thinking",
                "Responsive Design", "UX Writing", "Interaction Design", "Visual Design", "User-Centered Design"
            ],
            "product_manager": [
                "Product Strategy", "Roadmap Development", "User Stories", "Market Research", "Competitive Analysis",
                "Agile/Scrum", "KPIs", "Analytics", "A/B Testing", "Stakeholder Management", "Backlog Prioritization",
                "Product Lifecycle", "Go-to-Market Strategy", "Product Launch", "User Feedback", "SQL", "JIRA", "Confluence"
            ],
            "devops_engineer": [
                "CI/CD", "Infrastructure as Code", "Docker", "Kubernetes", "Jenkins", "Travis CI", "GitLab CI",
                "AWS", "Azure", "Google Cloud", "Terraform", "Ansible", "Puppet", "Chef", "Monitoring", "Logging",
                "Shell Scripting", "Python", "Linux/Unix", "Networking", "Security", "Troubleshooting", "Automation"
            ]
        }
        
        # Section content guidance
        self.section_guidance = {
            "Professional Summary": {
                "description": "A brief 3-4 sentence overview of your career, skills, and value.",
                "tips": [
                    "Keep it concise (50-100 words)",
                    "Highlight years of experience",
                    "Mention key technical skills relevant to the role",
                    "Include 1-2 notable achievements",
                    "Tailor to the specific job description"
                ],
                "example": "Results-driven Software Engineer with 5+ years of experience building scalable web applications. Proficient in Python, Django, and AWS cloud services. Reduced API response time by 40% and implemented CI/CD pipelines that cut deployment time in half. Seeking to leverage technical expertise to drive innovation at {Company Name}."
            },
            "Skills": {
                "description": "A section highlighting your technical and soft skills.",
                "tips": [
                    "Group skills by category (Programming Languages, Frameworks, Tools, etc.)",
                    "List most relevant skills first",
                    "Include proficiency level if appropriate",
                    "Focus on keywords from the job description",
                    "Be honest about your skill levels"
                ],
                "example": "Technical Skills:\n- Languages: Python (Expert), JavaScript (Proficient), SQL (Proficient), Java (Familiar)\n- Frameworks: Django, React, Flask\n- Tools: Git, Docker, Kubernetes, AWS (EC2, S3, Lambda)\n\nSoft Skills:\n- Team Collaboration\n- Problem Solving\n- Communication\n- Project Management"
            },
            "Work Experience": {
                "description": "Detailed information about your previous roles and accomplishments.",
                "tips": [
                    "Use reverse chronological order (most recent first)",
                    "Include company name, location, job title, and dates",
                    "Focus on achievements rather than responsibilities",
                    "Quantify results when possible (%, $, time saved)",
                    "Use action verbs (Developed, Implemented, Optimized)",
                    "Include 3-5 bullet points per role"
                ],
                "example": "Senior Software Engineer | TechCorp Inc., San Francisco, CA | Jan 2020 - Present\n- Architected and implemented a microservices-based API that processes 1M+ requests daily\n- Reduced database query time by 60% through optimization and indexing\n- Led a team of 4 developers to deliver a major feature on time and under budget\n- Implemented automated testing that improved code coverage from 65% to 92%"
            },
            "Education": {
                "description": "Information about your academic background.",
                "tips": [
                    "Include degree, institution, location, and graduation year",
                    "Add GPA if it's impressive (3.5+)",
                    "List relevant coursework for recent graduates",
                    "Include academic honors or achievements",
                    "Add certifications in this section or create a separate one"
                ],
                "example": "Bachelor of Science in Computer Science\nUniversity of California, Berkeley | 2018\nGPA: 3.7/4.0\nRelevant Coursework: Data Structures, Algorithms, Machine Learning, Database Systems"
            }
        }

    def get_resume_templates(self):
        """
        Return a list of available resume templates with descriptions
        """
        template_info = []
        for name, template in self.resume_templates.items():
            template_info.append(f"{name.capitalize()}: {template['description']}")
        return template_info

    def get_template_sections(self, template_name):
        """
        Return the sections for a specific template
        """
        template_name = template_name.lower()
        if template_name in self.resume_templates:
            return self.resume_templates[template_name]["sections"]
        else:
            return ["Template not found. Available templates: " + ", ".join(self.resume_templates.keys())]
    
    def get_ats_keywords(self, role):
        """
        Return ATS keywords for a specific role
        """
        role = role.lower().replace(" ", "_")
        if role in self.ats_keywords:
            return self.ats_keywords[role]
        else:
            return ["Role not found. Available roles: " + ", ".join([r.replace("_", " ") for r in self.ats_keywords.keys()])]
    
    def get_section_guidance(self, section_name):
        """
        Return guidance for a specific resume section
        """
        if section_name in self.section_guidance:
            guidance = self.section_guidance[section_name]
            result = f"--- {section_name} ---\n"
            result += f"Description: {guidance['description']}\n\n"
            result += "Tips:\n"
            for tip in guidance['tips']:
                result += f"- {tip}\n"
            result += f"\nExample:\n{guidance['example']}"
            return result
        else:
            return f"Section not found. Available sections: {', '.join(self.section_guidance.keys())}"
    
    def analyze_resume_query(self, query):
        """
        Analyze a user query and return relevant resume assistance
        """
        query = query.lower()
        
        # Checking if the query is about resume templates
        if any(word in query for word in ["template", "format", "layout", "structure"]):
            return f"Here are some resume templates you might consider:\n\n" + "\n\n".join(self.get_resume_templates())
        
        # Checking if the query is about ATS or keywords
        if any(word in query for word in ["ats", "keyword", "applicant tracking", "scanner"]):
            roles = []
            for role in self.ats_keywords.keys():
                role_name = role.replace("_", " ")
                if role_name in query:
                    roles.append(role)
            
            if roles:
                response = "Here are important keywords for ATS optimization:\n\n"
                for role in roles:
                    keywords = self.get_ats_keywords(role)
                    response += f"For {role.replace('_', ' ')} roles:\n- " + "\n- ".join(keywords[:10]) + "\n\nInclude these keywords naturally throughout your resume, especially in your skills section and work experience."
                return response
            else:
                return "I can provide ATS keywords for specific roles. Please specify which role you're interested in (e.g., software engineer, data scientist, UX designer, product manager, or DevOps engineer)."
        
        # Checking if the query is about specific sections
        for section in self.section_guidance.keys():
            if section.lower() in query:
                return self.get_section_guidance(section)
        
        # Default response for general resume questions
        return "I can help with your resume in several ways:\n\n1. Recommend templates and formats\n2. Provide ATS keywords for specific roles\n3. Give guidance on specific sections (Professional Summary, Skills, Work Experience, Education)\n\nWhat specific aspect of your resume would you like help with?"


# Creating an instance of the ResumeAssistant
resume_assistant = ResumeAssistant()

In [17]:
# Function to integrate the resume assistant with the conversation manager
def integrate_resume_assistant(conversation_manager, user_input):
    """
    Properly integrates the resume assistant with the conversation manager
    
    Args:
        conversation_manager: The conversation manager containing history
        user_input: The current user input
        
    Returns:
        Response from resume assistant, taking conversation context into account
    """
    # Getting relevant conversation history
    history = conversation_manager.conversation_history
    
    # Analyzing the query using resume assistant
    response = resume_assistant.analyze_resume_query(user_input)
    
    # Updating conversation manager with the exchange
    conversation_manager.add_exchange(user_input, response)
    
    # Set topic to resume
    conversation_manager.topic = "resume"
    
    return response

In [18]:
# # Testing the resume assistant with example queries
# test_queries = [
#     "What resume template should I use?",
#     "What are good ATS keywords for a software engineer?",
#     "How should I write my professional summary?",
#     "What should I include in my skills section?",
#     "How do I format my work experience?"
# ]

# print("Testing Resume Assistant Module:\n")
# for query in test_queries:
#     print(f"User: {query}")
#     response = integrate_resume_assistant(conversation_manager, query)
#     print(f"Bot: {response}\n")

### Cover Letter Module Implementation

In [19]:
class CoverLetterAssistant:
    def __init__(self):
        # Initializing cover letter templates
        self.letter_templates = {
            "standard": {
                "description": "Traditional format suitable for most industries.",
                "structure": ["Header", "Date", "Employer Address", "Greeting", "Opening Paragraph", "Body Paragraphs", "Closing Paragraph", "Signature"]
            },
            "modern": {
                "description": "Contemporary format with a more personal tone, good for startups and creative roles.",
                "structure": ["Header", "Date", "Greeting", "Hook/Opening", "Skills Alignment", "Company Connection", "Call to Action", "Signature"]
            },
            "technical": {
                "description": "Format emphasizing technical skills and achievements, ideal for tech positions.",
                "structure": ["Header", "Date", "Greeting", "Technical Introduction", "Project Highlights", "Technical Skills Alignment", "Company Technology Connection", "Call to Action", "Signature"]
            }
        }
        
        # Section content guidance
        self.section_guidance = {
            "Header": {
                "description": "Your contact information at the top of the letter.",
                "tips": [
                    "Include your full name, phone number, email, and LinkedIn profile",
                    "Use the same header format as your resume for consistency",
                    "Keep it professional and clean"
                ],
                "example": "John Smith\n123-456-7890 | john.smith@email.com | linkedin.com/in/johnsmith"
            },
            "Opening Paragraph": {
                "description": "Introduction that hooks the reader and states your interest in the position.",
                "tips": [
                    "Mention the specific position you're applying for",
                    "Include how you discovered the opening",
                    "Express enthusiasm for the role",
                    "Add a brief statement about why you're a good fit"
                ],
                "example": "As a passionate software developer with 5+ years of experience building scalable web applications, I was excited to see the Senior Developer position at TechCorp. After reading about your company's innovative work in AI and commitment to sustainable technology, I am eager to bring my expertise in Python and cloud architecture to your team."
            },
            "Body Paragraphs": {
                "description": "Detailed explanation of your qualifications and alignment with the job requirements.",
                "tips": [
                    "Focus on 2-3 key qualifications relevant to the job description",
                    "Provide specific examples and achievements with metrics",
                    "Connect your experience directly to the company's needs",
                    "Show how you can add value to the organization"
                ],
                "example": "At XYZ Company, I led the development of a microservices architecture that reduced system downtime by 40% and improved response times by 25%. This experience directly relates to your need for a developer who can optimize your customer-facing applications.\n\nAdditionally, my work implementing automated testing frameworks reduced bug rates by 35% while allowing our team to deploy twice as frequently. I understand that TechCorp values continuous delivery, and I'm excited to bring these DevOps practices to your development workflow."
            },
            "Closing Paragraph": {
                "description": "Conclusion that reiterates interest and includes a call to action.",
                "tips": [
                    "Reiterate your enthusiasm for the position",
                    "Summarize why you're a strong candidate",
                    "Include a clear call to action",
                    "Thank the reader for their consideration"
                ],
                "example": "I'm excited about the possibility of bringing my technical expertise and passion for innovation to TechCorp. I would welcome the opportunity to discuss how my background aligns with your team's goals. Thank you for considering my application, and I look forward to speaking with you soon."
            }
        }
        
        # Job description analysis tips
        self.job_analysis_tips = [
            "Identify key technical skills and tools mentioned",
            "Note repeated words or phrases (these are likely priorities)",
            "Look for 'must-have' versus 'nice-to-have' qualifications",
            "Identify company values and culture indicators",
            "Pay attention to specific responsibilities and projects mentioned",
            "Look for problems the company is trying to solve"
        ]
        
        # Common mistakes to avoid
        self.common_mistakes = [
            "Using a generic template without customization",
            "Repeating your resume word-for-word",
            "Writing more than one page",
            "Focusing too much on what you want rather than what you can offer",
            "Including irrelevant experience or skills",
            "Using an overly formal or stiff tone",
            "Failing to address specific job requirements",
            "Grammar and spelling errors"
        ]
    
    def get_letter_templates(self):
        """
        Return a list of available cover letter templates with descriptions
        """
        template_info = []
        for name, template in self.letter_templates.items():
            template_info.append(f"{name.capitalize()}: {template['description']}")
        return template_info
    
    def get_template_structure(self, template_name):
        """
        Return the structure for a specific template
        """
        template_name = template_name.lower()
        if template_name in self.letter_templates:
            return self.letter_templates[template_name]["structure"]
        else:
            return ["Template not found. Available templates: " + ", ".join(self.letter_templates.keys())]
    
    def get_section_guidance(self, section_name):
        """
        Return guidance for a specific cover letter section
        """
        if section_name in self.section_guidance:
            guidance = self.section_guidance[section_name]
            result = f"--- {section_name} ---\n"
            result += f"Description: {guidance['description']}\n\n"
            result += "Tips:\n"
            for tip in guidance['tips']:
                result += f"- {tip}\n"
            result += f"\nExample:\n{guidance['example']}"
            return result
        else:
            return f"Section not found. Available sections: {', '.join(self.section_guidance.keys())}"
    
    def get_job_analysis_tips(self):
        """
        Return tips for analyzing job descriptions
        """
        result = "Tips for Analyzing Job Descriptions:\n\n"
        for i, tip in enumerate(self.job_analysis_tips, 1):
            result += f"{i}. {tip}\n"
        return result
    
    def get_common_mistakes(self):
        """
        Return common cover letter mistakes to avoid
        """
        result = "Common Cover Letter Mistakes to Avoid:\n\n"
        for i, mistake in enumerate(self.common_mistakes, 1):
            result += f"{i}. {mistake}\n"
        return result
    
    def provide_personalization_tips(self):
        """
        Return tips for personalizing a cover letter
        """
        tips = [
            "Research the company's mission, values, recent projects, and culture",
            "Find the hiring manager's name if possible (avoid 'To Whom It May Concern')",
            "Reference specific company projects or initiatives that excite you",
            "Explain why you want to work at this specific company, not just any company",
            "Connect your personal values to the company's mission",
            "Mention any connections you have at the company (if applicable)",
            "Use the company's terminology from their website or job description"
        ]
        
        result = "How to Personalize Your Cover Letter:\n\n"
        for i, tip in enumerate(tips, 1):
            result += f"{i}. {tip}\n"
        return result
    
    def analyze_cover_letter_query(self, query):
        """
        Analyze a user query and return relevant cover letter assistance
        """
        query = query.lower()
        
        # Checking if the query is about cover letter templates
        if any(word in query for word in ["template", "format", "structure"]):
            return f"Here are some cover letter templates you might consider:\n\n" + "\n\n".join(self.get_letter_templates())
        
        # Checking if the query is about job description analysis
        if any(phrase in query for phrase in ["job description", "analyze job", "job posting", "job ad"]):
            return self.get_job_analysis_tips()
        
        # Checking if the query is about common mistakes
        if any(phrase in query for phrase in ["mistake", "error", "avoid", "don't do"]):
            return self.get_common_mistakes()
        
        # Checking if the query is about personalization
        if any(word in query for word in ["personalize", "customize", "tailor", "specific"]):
            return self.provide_personalization_tips()
        
        # Checking if the query is about specific sections
        for section in self.section_guidance.keys():
            section_lower = section.lower()
            if section_lower in query or section_lower.replace(" ", "") in query.replace(" ", ""):
                return self.get_section_guidance(section)
        
        # Default response for general cover letter questions
        return "I can help with your cover letter in several ways:\n\n1. Recommend templates and formats\n2. Provide guidance on specific sections (Opening Paragraph, Body Paragraphs, etc.)\n3. Share tips on personalizing your letter\n4. Help analyze job descriptions\n5. Highlight common mistakes to avoid\n\nWhat specific aspect of your cover letter would you like help with?"

# Creating an instance of the CoverLetterAssistant
cover_letter_assistant = CoverLetterAssistant()


In [20]:
# Function to integrate the cover letter assistant with the conversation manager
def integrate_cover_letter_assistant(conversation_manager, user_input):
    """
    Integrate the cover letter assistant with the conversation manager
    
    Args:
        conversation_manager: The conversation manager containing history
        user_input: The current user input
        
    Returns:
        Response from cover letter assistant, taking conversation context into account
    """
    # Getting relevant conversation history
    history = conversation_manager.conversation_history
    
    # Analyzing the query using cover letter assistant
    response = cover_letter_assistant.analyze_cover_letter_query(user_input)
    
    # Updating conversation manager with the exchange
    conversation_manager.add_exchange(user_input, response)
    
    # Setting the topic to cover letter
    conversation_manager.topic = "cover_letter"
    
    return response

In [21]:
# # Testing the cover letter assistant with example queries
# test_queries = [
#     "What cover letter template should I use?",
#     "How do I analyze a job description?",
#     "What should I include in my opening paragraph?",
#     "How do I personalize my cover letter?",
#     "What are common cover letter mistakes to avoid?"
# ]

# # For testing without a conversation manager
# print("Testing Cover Letter Assistant Module:\n")
# for query in test_queries:
#     print(f"User: {query}")
#     response = cover_letter_assistant.analyze_cover_letter_query(query)
#     print(f"Bot: {response}\n")

# # For testing with the conversation manager integration
# conversation_manager = ConversationManager(tokenizer)
# print("Testing Cover Letter Assistant with Conversation Manager:\n")
# for query in test_queries:
#     print(f"User: {query}")
#     response = integrate_cover_letter_assistant(conversation_manager, query)
#     print(f"Bot: {response}\n")
# print("Conversation Summary after Cover Letter Assistant Testing:")
# print(conversation_manager.get_conversation_summary())

### Interview Preparation Module Implementation

In [22]:
class InterviewPreparationAssistant:
    def __init__(self):
        # Common interview questions by category
        self.interview_questions = {
            "behavioral": [
                "Tell me about yourself.",
                "Why are you interested in working for our company?",
                "Describe a challenging work situation and how you overcame it.",
                "Tell me about a time you demonstrated leadership skills.",
                "How do you handle tight deadlines or pressure?",
                "Describe a situation where you had to work with a difficult team member.",
                "Tell me about a time you failed and what you learned from it.",
                "How do you prioritize your work when you have multiple deadlines?",
                "Tell me about a time you went above and beyond at work.",
                "How do you handle criticism or feedback?"
            ],
            "technical_general": [
                "What programming languages are you proficient in?",
                "Explain your approach to debugging a complex issue.",
                "How do you ensure your code is maintainable and readable?",
                "Describe your experience with version control systems.",
                "How do you stay updated with the latest technologies?",
                "Explain the difference between object-oriented and functional programming.",
                "What is your approach to testing your code?",
                "Tell me about a challenging technical problem you solved recently.",
                "How would you optimize a slow-performing application?",
                "Describe your experience with agile development methodologies."
            ],
            "software_engineer": [
                "What is the time and space complexity of this algorithm?",
                "Explain how you would implement a cache system.",
                "How would you design a scalable web application?",
                "Explain the concept of recursion and provide an example.",
                "What are the key considerations when designing a RESTful API?",
                "How do you handle race conditions in concurrent programming?",
                "Explain the difference between a stack and a queue.",
                "How would you implement a linked list?",
                "Describe the process of database normalization.",
                "How would you optimize a SQL query that's running slowly?"
            ],
            "data_scientist": [
                "Explain the difference between supervised and unsupervised learning.",
                "How do you handle missing data in a dataset?",
                "Explain the bias-variance tradeoff.",
                "What evaluation metrics would you use for a classification problem?",
                "How would you detect outliers in a dataset?",
                "Explain the concept of overfitting and how to prevent it.",
                "How would you approach a text classification problem?",
                "What is the difference between bagging and boosting?",
                "Explain how a random forest algorithm works.",
                "How would you validate the results of your model?"
            ],
            "product_manager": [
                "How do you prioritize features in a product backlog?",
                "Describe how you would gather user requirements for a new product.",
                "How do you measure the success of a product?",
                "Tell me about a time you had to make a difficult product decision.",
                "How do you balance user needs with business goals?",
                "Describe your approach to creating a product roadmap.",
                "How do you handle stakeholders with conflicting priorities?",
                "What metrics would you track for a mobile app?",
                "How do you incorporate user feedback into product development?",
                "Describe how you would launch a new product feature."
            ]
        }
        
        # Tips for different interview types
        self.interview_type_tips = {
            "phone": [
                "Find a quiet location with good reception",
                "Have your resume, the job description, and company notes in front of you",
                "Keep a glass of water nearby",
                "Smile while speaking to convey enthusiasm",
                "Stand up to project confidence in your voice",
                "Be concise - phone interviews typically last 30 minutes",
                "Prepare 2-3 questions to ask the interviewer"
            ],
            "video": [
                "Test your technology beforehand (camera, microphone, internet)",
                "Choose a clean, professional background",
                "Dress professionally (full outfit, not just the top)",
                "Look at the camera, not the screen, to maintain eye contact",
                "Close other applications and turn off notifications",
                "Position your camera at eye level",
                "Ensure proper lighting (light source in front of you, not behind)",
                "Have a backup plan if technical issues arise"
            ],
            "in_person": [
                "Research the office location and plan your route",
                "Arrive 10-15 minutes early",
                "Bring multiple copies of your resume",
                "Dress appropriately for the company culture (when in doubt, business professional)",
                "Be courteous to everyone you meet, including receptionists",
                "Prepare for lunch or coffee interviews where applicable",
                "Bring a notepad and pen to take notes",
                "Prepare for multiple rounds of interviews in one day"
            ],
            "technical": [
                "Practice coding on a whiteboard or shared document",
                "Verbalize your thought process while solving problems",
                "Ask clarifying questions before jumping into solutions",
                "Start with a naive approach, then optimize",
                "Prepare to explain your previous technical projects in detail",
                "Review fundamentals like data structures and algorithms",
                "Research common technical questions for your specific role",
                "Be prepared to discuss your technical strengths and weaknesses"
            ]
        }
        
        # STAR method explanation
        self.star_method = {
            "description": "The STAR method is a structured way to answer behavioral interview questions by describing a specific Situation, Task, Action, and Result.",
            "components": {
                "Situation": "Describe the context or background. Where? When? With whom?",
                "Task": "Explain the challenge, assignment, or problem you faced. What needed to be done? Why?",
                "Action": "Detail the specific actions you took. Focus on YOUR contribution, even if it was a team effort.",
                "Result": "Share the outcomes of your actions. Quantify results when possible. Include what you learned if applicable."
            },
            "example": {
                "question": "Tell me about a time you had to meet a tight deadline.",
                "answer": "Situation: At XYZ Company, our team was developing a new feature for our primary client's e-commerce platform. Two weeks before delivery, we discovered a major security vulnerability.\n\nTask: As the lead developer, I needed to redesign the authentication system while still meeting the original deadline, as the client had a major marketing campaign planned.\n\nAction: I prioritized the work by severity, redistributed non-critical tasks to team members, created a detailed timeline, and worked additional hours to focus on the core security fixes. I also maintained daily communication with the client to set expectations.\n\nResult: We delivered the project on time with all security issues resolved. The client was extremely satisfied, and our solution was so effective that we implemented it across other projects, reducing similar vulnerabilities by 40%."
            }
        }
        
        # Common salary negotiation tips
        self.salary_negotiation_tips = [
            "Research industry salary ranges for your position and location",
            "Know your minimum acceptable salary and ideal target",
            "Wait for the employer to bring up compensation first if possible",
            "Consider the entire compensation package, not just salary",
            "Respond to offers with gratitude before negotiating",
            "Use your skills, experience, and achievements as leverage",
            "Avoid giving specific numbers early in the interview process",
            "Practice your negotiation conversation beforehand",
            "Get the final offer in writing",
            "Be prepared to walk away if the offer doesn't meet your needs"
        ]
    
    def get_interview_questions(self, category):
        """
        Return interview questions for a specific category
        """
        if category in self.interview_questions:
            return self.interview_questions[category]
        else:
            available_categories = list(self.interview_questions.keys())
            return [f"Category not found. Available categories: {', '.join(available_categories)}"]
    
    def get_random_question(self, category):
        """
        Return a random interview question from a specific category
        """
        if category in self.interview_questions:
            return random.choice(self.interview_questions[category])
        else:
            available_categories = list(self.interview_questions.keys())
            return f"Category not found. Available categories: {', '.join(available_categories)}"
    
    def get_interview_type_tips(self, interview_type):
        """
        Return tips for a specific interview type
        """
        interview_type = interview_type.lower()
        if interview_type in self.interview_type_tips:
            tips = self.interview_type_tips[interview_type]
            result = f"Tips for {interview_type.capitalize()} Interviews:\n\n"
            for i, tip in enumerate(tips, 1):
                result += f"{i}. {tip}\n"
            return result
        else:
            available_types = list(self.interview_type_tips.keys())
            return f"Interview type not found. Available types: {', '.join(available_types)}"
    
    def get_star_method_explanation(self):
        """
        Return explanation of the STAR method for answering behavioral questions
        """
        star = self.star_method
        result = f"The STAR Method for Behavioral Interview Questions\n\n"
        result += f"{star['description']}\n\n"
        
        result += "Components:\n"
        for component, description in star['components'].items():
            result += f"- {component}: {description}\n"
        
        result += f"\nExample Question: {star['example']['question']}\n\n"
        result += f"Example Answer:\n{star['example']['answer']}"
        
        return result
    
    def get_salary_negotiation_tips(self):
        """
        Return tips for salary negotiation
        """
        result = "Salary Negotiation Tips:\n\n"
        for i, tip in enumerate(self.salary_negotiation_tips, 1):
            result += f"{i}. {tip}\n"
        return result
    
    def conduct_mock_interview(self, role, num_questions=5):
        """
        Conduct a mock interview with questions relevant to the specified role
        """
        questions = []
        
        # Some behavioral questions
        behavioral_questions = random.sample(self.interview_questions["behavioral"], min(3, num_questions))
        questions.extend(behavioral_questions)
        
        # Adding technical questions based on role
        if role.lower() in ["software_engineer", "software engineer", "developer", "programmer"]:
            technical_questions = random.sample(self.interview_questions["software_engineer"], min(2, num_questions - len(questions)))
            questions.extend(technical_questions)
        elif role.lower() in ["data_scientist", "data scientist", "ml engineer", "machine learning"]:
            technical_questions = random.sample(self.interview_questions["data_scientist"], min(2, num_questions - len(questions)))
            questions.extend(technical_questions)
        elif role.lower() in ["product_manager", "product", "pm"]:
            role_questions = random.sample(self.interview_questions["product_manager"], min(2, num_questions - len(questions)))
            questions.extend(role_questions)
        
        # Adding general technical questions
        if len(questions) < num_questions:
            general_questions = random.sample(self.interview_questions["technical_general"], min(num_questions - len(questions), len(self.interview_questions["technical_general"])))
            questions.extend(general_questions)
        
        # Setting limit to requested number of questions
        questions = questions[:num_questions]
        
        result = f"Mock Interview Questions for {role} Role:\n\n"
        for i, question in enumerate(questions, 1):
            result += f"{i}. {question}\n\n"
        
        result += "To conduct this mock interview effectively:\n"
        result += "1. Record yourself answering each question\n"
        result += "2. Set a timer for 1-2 minutes per question\n"
        result += "3. Review your answers afterward for clarity and completeness\n"
        result += "4. For behavioral questions, practice using the STAR method\n"
        result += "5. For technical questions, explain your thought process clearly"
        
        return result
    
    def analyze_interview_query(self, query):
        """
        Analyze a user query and return relevant interview preparation assistance
        """
        query = query.lower()
        
        # Checking if the query is about specific interview types
        for interview_type in self.interview_type_tips.keys():
            if interview_type in query:
                return self.get_interview_type_tips(interview_type)
        
        # Checking if the query is about the STAR method
        if "star" in query or "behavioral" in query and "answer" in query:
            return self.get_star_method_explanation()
        
        # Checking if the query is about salary negotiation
        if "salary" in query or "negotiat" in query or "compensation" in query:
            return self.get_salary_negotiation_tips()
        
        # Checking if the query is about a mock interview
        if "mock" in query or "practice interview" in query:
            # Extract role if mentioned
            role = "general"
            for potential_role in ["software engineer", "software_engineer", "developer", "data scientist", 
                                   "data_scientist", "product manager", "product_manager"]:
                if potential_role in query:
                    role = potential_role
                    break
            
            return self.conduct_mock_interview(role)
        
        # Checking if the query is about specific question categories
        for category in self.interview_questions.keys():
            if category in query.replace(" ", "_"):
                return "\n\n".join([f"Q: {q}" for q in self.get_interview_questions(category)[:5]]) + "\n\n(Showing 5 sample questions)"
        
        # Default response for general interview questions
        return "I can help with your interview preparation in several ways:\n\n1. Provide common interview questions for different roles\n2. Offer tips for phone, video, in-person, or technical interviews\n3. Explain the STAR method for answering behavioral questions\n4. Conduct a mock interview with relevant questions\n5. Share salary negotiation tips\n\nWhat specific aspect of interview preparation would you like help with?"

# Creating an instance of the InterviewPreparationAssistant
interview_assistant = InterviewPreparationAssistant()

In [23]:
# Function to integrate the interview assistant with the conversation manager
def integrate_interview_assistant(conversation_manager, user_input):
    """
    Integrate the interview preparation assistant with the conversation manager
    
    Args:
        conversation_manager: The conversation manager containing history
        user_input: The current user input
        
    Returns:
        Response from interview assistant, taking conversation context into account
    """
    # Getting the relevant conversation history
    history = conversation_manager.conversation_history
    
    # Analyzing the query using interview assistant
    response = interview_assistant.analyze_interview_query(user_input)
    
    # Updating conversation manager with the exchange
    conversation_manager.add_exchange(user_input, response)
    
    # Setting the topic to interview
    conversation_manager.topic = "interview"
    
    return response

In [24]:
# # Testing the interview assistant with some example queries
# test_queries = [
#     "What are some common behavioral interview questions?",
#     "How should I prepare for a technical interview?",
#     "Can you explain the STAR method?",
#     "Can you conduct a mock interview for a software engineer?",
#     "What are some tips for salary negotiation?"
# ]

# # For testing without a conversation manager
# print("Testing Interview Preparation Assistant Module:\n")
# for query in test_queries:
#     print(f"User: {query}")
#     response = interview_assistant.analyze_interview_query(query)
#     print(f"Bot: {response}\n")

# # For testing with the conversation manager integration
# conversation_manager = ConversationManager(tokenizer)
# print("Testing Interview Preparation Assistant with Conversation Manager:\n")
# for query in test_queries:
#     print(f"User: {query}")
#     response = integrate_interview_assistant(conversation_manager, query)
#     print(f"Bot: {response}\n")
# print("Conversation Summary after Interview Preparation Assistant Testing:")
# print(conversation_manager.get_conversation_summary())

### Integration and UI

In [25]:
class JobSeekerChatbot:
    def __init__(self, model, tokenizer, device):
        """
        Initialize the integrated chatbot with all modules
        """
        self.model = model
        self.tokenizer = tokenizer
        self.device = device
        
        # Initializing the conversation manager
        self.conversation_manager = ConversationManager(tokenizer)
        
        # Initializing all specialized modules
        self.resume_assistant = ResumeAssistant()
        self.cover_letter_assistant = CoverLetterAssistant()
        self.interview_assistant = InterviewPreparationAssistant()
        
        # Welcome message
        self.welcome_message = """
Welcome to the Job Seeker Assistant Chatbot! 

I can help you with:
1. 📄 Resume building and optimization
2. ✉️ Cover letter writing and customization
3. 🎯 Interview preparation and practice

What would you like help with today?
        """
    
    def detect_intent(self, user_input):
        """
        Detect the user's intent to route to appropriate module
        """
        user_input_lower = user_input.lower()
        
        # Checking for resume-related keywords
        if any(word in user_input_lower for word in ['resume', 'cv', 'experience', 'skill', 'qualification']):
            return "resume"
        
        # Checking for cover letter-related keywords
        elif any(word in user_input_lower for word in ['cover letter', 'application letter', 'introduction letter']):
            return "cover_letter"
        
        # Checking for interview-related keywords
        elif any(word in user_input_lower for word in ['interview', 'question', 'mock', 'behavioral', 'technical', 'salary']):
            return "interview"
        
        # If there's a topic already set, continue with it
        elif self.conversation_manager.topic:
            return self.conversation_manager.topic
        
        # Default to general conversation
        else:
            return "general"
    
    def get_response(self, user_input):
        """
        Generate a response based on detected intent
        """
        # Detecting the user's intent
        intent = self.detect_intent(user_input)
        
        # Route to appropriate module based on intent
        if intent == "resume":
            response = self.resume_assistant.analyze_resume_query(user_input)
            self.conversation_manager.topic = "resume"
        
        elif intent == "cover_letter":
            response = self.cover_letter_assistant.analyze_cover_letter_query(user_input)
            self.conversation_manager.topic = "cover_letter"
        
        elif intent == "interview":
            response = self.interview_assistant.analyze_interview_query(user_input)
            self.conversation_manager.topic = "interview"
        
        else:
            # General conversation using the language model
            response = self.get_model_response(user_input)
        
        # Updating conversation history
        self.conversation_manager.add_exchange(user_input, response)
        
        return response
    
    def get_model_response(self, user_input):
        """
        Generate a response using the language model for general conversation
        """
        try:
            # Encoding the user input
            new_user_input_ids = self.tokenizer.encode(user_input + self.tokenizer.eos_token, return_tensors='pt').to(device)
            
            # Getting conversation history
            chat_history_ids = self.conversation_manager.get_token_history()
            
            # If there's chat history, combining it with the new input
            if chat_history_ids is not None:
                # Reshape history to match the new input's batch dimension
                if chat_history_ids.dim() == 1:
                    chat_history_ids = chat_history_ids.unsqueeze(0)
                
                bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1)
            else:
                bot_input_ids = new_user_input_ids
            
            # Creating attention mask
            attention_mask = torch.ones(bot_input_ids.shape, dtype=torch.long, device=device)
            
            # Generating response
            output_ids = self.model.generate(
                bot_input_ids,
                attention_mask=attention_mask,
                max_length=bot_input_ids.shape[-1] + 100,
                pad_token_id=self.tokenizer.pad_token_id,
                no_repeat_ngram_size=2,
                do_sample=True,
                top_p=TOP_P,
                top_k=40,
                temperature=TEMPERATURE,
                repetition_penalty=1.2
            )
            
            # Extracting the response
            response_ids = output_ids[0, bot_input_ids.shape[-1]:]
            response = self.tokenizer.decode(response_ids, skip_special_tokens=True)
            
            # If response is empty or too generic
            if not response or response.isspace():
                response = "I'm here to help with your job search. Could you tell me more about what you need help with? I can assist with resumes, cover letters, or interview preparation."
            
            return response
            
        except Exception as e:
            return f"I apologize, but I encountered an error. Could you rephrase your question or ask about resumes, cover letters, or interview preparation? (Error: {str(e)})"
    
    def reset_conversation(self):
        """
        Reset the conversation history
        """
        self.conversation_manager.clear_history()
        return "Conversation has been reset. How can I help you with your job search today?"

In [26]:
# Creating an interactive UI for the chatbot
def create_chatbot_ui(chatbot):
    """
    Create an interactive UI for the chatbot using ipywidgets
    """
    # Create widgets
    output = widgets.Output(layout={'border': '1px solid black', 'width': 'auto', 'height': '400px', 'overflow_y': 'auto'})
    text_input = widgets.Text(
        value='',
        placeholder='Type your message here...',
        description='',
        layout=widgets.Layout(width='80%')
    )
    send_button = widgets.Button(
        description='Send',
        disabled=False,
        button_style='primary',
        tooltip='Send message',
        icon='paper-plane',
        layout=widgets.Layout(width='18%')
    )
    reset_button = widgets.Button(
        description='Reset Chat',
        disabled=False,
        button_style='warning',
        tooltip='Reset conversation',
        icon='refresh',
        layout=widgets.Layout(width='20%')
    )
    end_button = widgets.Button(
        description='End Conversation',
        disabled=False,
        button_style='danger',
        tooltip='End the conversation and shutdown the chatbot',
        icon='stop',
        layout=widgets.Layout(width='25%')
    )
    
    input_box = widgets.HBox([text_input, send_button])
    button_box = widgets.HBox([reset_button, end_button])
    
    # Displaying welcome message
    with output:
        print("Bot: " + chatbot.welcome_message)
    
    # Defining callbacks
    def on_button_clicked(b):
        user_message = text_input.value
        text_input.value = ''
        
        if not user_message.strip():
            return  # Ignores empty messages
        
        # Displaying user message
        with output:
            print(f"You: {user_message}")
        
        # Typing indicator
        with output:
            print("Bot: ", end="")
            time.sleep(0.5)
            clear_output(wait=True)
        
        # Getting and displaying bot response
        response = chatbot.get_response(user_message)
        
        with output:
            print(f"You: {user_message}")
            print(f"Bot: {response}")
    
    def on_text_enter(sender):
        on_button_clicked(None)
    
    def on_reset_clicked(b):
        with output:
            clear_output()
            print("Bot: " + chatbot.reset_conversation())
    
    def on_end_clicked(b):
        # Disable all interactive elements
        text_input.disabled = True
        send_button.disabled = True
        reset_button.disabled = True
        end_button.disabled = True

        with output:
            print("Bot: Thank you for using the Job Seeker Assistant Chatbot. The conversation has ended.")
            print("Bot: To start a new conversation, please restart the notebook kernel and run all cells again.")
        
    # Connecting callbacks
    send_button.on_click(on_button_clicked)
    text_input.on_submit(on_text_enter)
    reset_button.on_click(on_reset_clicked)
    end_button.on_click(on_end_clicked)
    
    # Creating and returning the complete UI
    return widgets.VBox([output, input_box, button_box])

In [27]:
# Initializing the chatbot and UI
def initialize_job_seeker_chatbot():
    """
    Initialize and display the job seeker chatbot
    """
    print("Initializing Job Seeker Assistant Chatbot...")
    
    # Create the integrated chatbot
    chatbot = JobSeekerChatbot(model, tokenizer, device)
    
    # Create and display the UI
    ui = create_chatbot_ui(chatbot)
    display(ui)
    
    return chatbot, ui

In [28]:
chatbot, ui = initialize_job_seeker_chatbot()

Initializing Job Seeker Assistant Chatbot...
